In [9]:
import torch
from diffusers import DiffusionPipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import re
from huggingface_hub import login
from google.colab import userdata
login(
    token=userdata.get("HUGGINGFACE_KEY"),
    add_to_git_credential=True
)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer    = AutoTokenizer.from_pretrained("google/gemma-2-9b-it", token = userdata.get("HUGGINGFACE_KEY") ,device=device)
Gemma9bmodel = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it",token = userdata.get("HUGGINGFACE_KEY"))

Gemma9bmodel.to("cuda")
def storygeneration(instruction:str):
  # Story Prompt Generation
    thread = ''
    input_prompt = """
          ### Instruction:
          {}

          ### CONSTRAINTS:
          {}

          ### Story:
          {}

          """

    input_text = input_prompt.format(
            {instruction}, # instruction
            "- The Story should be simple so that kids can understand and short with a proper ending. - The ending should be satisfying and leave the main character in a happy place.         - Many bedtime stories end with a main character hugging a caregiver. This is a great way to send a child off to sleep.         - You want to engage the child and capture their imagination, but you don’t want them to have to expend too much brain power following the story. It’s a tricky skill to create vivid stories with simple language, but with practice you will be able to achieve this,        - Many classic bedtime stories end with the main character learning a valuable lesson or moral. Try to incorporate this into your tale as well, but write it in a way that is organic, not preachy", # input
            "", # output - leave this blank for generation!
        )

    input_ids  = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs    = Gemma9bmodel.generate(**input_ids, max_new_tokens=300)
    text       = tokenizer.decode(outputs[0])
    story_start_index = text.find("### Story:") + len("### Story:")
    story_text = text[story_start_index:].strip()
    return story_text

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
def generateillustrationtext(story):
  input_prompt = """
                ### Instruction:
                {}

                ### Story:
                {}

                ### Illustrations:
                {}

                """

  input_text = input_prompt.format(
          "You are an illustrator, read the following kids story and give 3 illustration descriptions for generating images which will be catchy for kids; there should be descriptions about appearance of characters not names", # instruction
          {story},
          "", # output - leave this blank for generation!
      )


  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  outputs = Gemma9bmodel.generate(**input_ids, max_new_tokens=300)

  illustrationtext = tokenizer.decode(outputs[0])


  story_start_index = illustrationtext.find("### Illustrations:") + len("### Illustrations:")
  story_text = illustrationtext[story_start_index:].strip()
  # Regular expression to extract descriptions
  matches = re.findall(r"\d+\.\s\*\*(.+?)\*\*", story_text)

  # Store descriptions in a list
  descriptions = [match.strip() for match in matches]

  return descriptions


def storyandillustrations(thread):
    story = storygeneration(thread)
    summary = generateillustrationtext(story)
    torch.cuda.empty_cache()
    return story, summary


def generateimageforillustrationtext(illustrationtext):

  from huggingface_hub import login
  from google.colab import userdata
  import random



  from huggingface_hub import InferenceClient
  client = InferenceClient("alvdansen/littletinies", token= userdata.get("HUGGINGFACE_KEY"))
  prompt = illustrationtext[0]
  # output is a PIL.Image object

  generated_images = []

  def get_image(prompt):
    image = client.text_to_image(prompt)
    return image


  #random_value = random.choice(illustrationtext)
  imageval     = get_image(prompt)

  return imageval


In [11]:
import base64
from io import BytesIO
from PIL import Image

def generate_storybook_html_with_image_variable(story_text, image, output_file="storybook.html"):
    """
    Generates a kids' storybook HTML page combining text and an image from a variable.

    Args:
        story_text (str): The story text to include in the HTML.
        image (PIL.Image): The image to include, provided as a PIL.Image object.
        output_file (str): The name of the output HTML file.
    """
    # Convert the image to Base64 string
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode()

    html_content = """
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Kenji's Story</title>
            <style>
                body {
                    font-family: 'Georgia', serif;
                    background-color: #fefae0;
                    margin: 0;
                    padding: 0;
                    color: #333;
                }
                .container {
                    max-width: 800px;
                    margin: 20px auto;
                    padding: 20px;
                    background-color: #fff;
                    border-radius: 10px;
                    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                }
                .story-text {
                    font-size: 18px;
                    line-height: 1.8;
                    text-align: justify;
                    margin-bottom: 20px;
                }
                .story-image {
                    text-align: center;
                    margin: 20px 0;
                }
                .story-image img {
                    max-width: 100%;
                    height: auto;
                    border-radius: 10px;
                }
            </style>
        </head>
        <body>
            <div class="container">
                <div class="story-text">"""+story_text+"""
                </div>
                <div class="story-image">
                    <img src='data:image/png;base64,"""+ img_base64 +"""' alt="Story Illustration">
                </div>
            </div>
        </body>
        </html>
        """
    # html_content = html_content.format(story_text, img_base64)


    # Write the HTML content to a file
    # with open(output_file, "w", encoding="utf-8") as file:
    #     file.write(html_content)

    return html_content

In [12]:

def curatestorybooktext(story):
  story, illustrationtext = storyandillustrations(story)
  imageval = generateimageforillustrationtext(illustrationtext)
  htmlstory = generate_storybook_html_with_image_variable(story, imageval, output_file="storybook.html")
  return htmlstory


In [6]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00


In [18]:
import gradio

gradio.Interface(curatestorybooktext, "text","html").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/12/01 16:47:24 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: connect: connection refused


<IPython.core.display.Javascript object>